In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas 
import os
from scripts.preprocessing import Obrobka
from scripts.preprocessing2 import Obrobka2
from torch.utils.data import Dataset, DataLoader
import numpy as np


class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.labels = torch.tensor(labels.values, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.relu = nn.ReLU()  # Activation function
        
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Hidden to output layer
        self.fc3 = nn.Linear(hidden_size, hidden_size)  # Hidden to output layer
        self.fc4 = nn.Linear(hidden_size, hidden_size)  # Hidden to output layer
        self.fc5 = nn.Linear(hidden_size, hidden_size)  # Hidden to output layer
        self.fc6 = nn.Linear(hidden_size, output_size)  # Hidden to output layer
        
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        x = self.relu(x)
        x = self.fc6(x)
        return x

obr = Obrobka()

# Create a model instance
input_size = 8  # Example: 4 input features
hidden_size = 100
output_size = 1  # Example: 3 output classes
model = SimpleNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example random input tensor (batch of 5 samples, each with 4 features)
zbior = CustomDataset(obr.X_train, obr.y_train)
dataloader = DataLoader(zbior, batch_size=50, shuffle=True)
model.train()
# Training loop (1 epoch for simplicity)
for _ in range(50):
    
    for features, labels in dataloader:

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(features)  # Forward pass
        #print((torch.sigmoid(outputs)>0.5).float)
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    #print(f"Loss: {loss.item()}")

zbior2 = CustomDataset(obr.X_test, obr.y_test)
dataloader2 = DataLoader(zbior, batch_size=4, shuffle=True)
model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for features, labels in dataloader2:
        
        outputs = model(features)  # Przepuść dane przez model
        predictions = (outputs > 0.5).float()  # Próg 0.5 dla klasyfikacji binarnej

        all_predictions.extend(predictions.cpu().numpy())  # Konwersja na NumPy
        all_labels.extend(labels.cpu().numpy())


accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
print(f"Test Accuracy: {accuracy:.4f}")

obr2 = Obrobka2()
dane_koncowe = obr2.train_float_data
#tensor_koncowy = torch.tensor(dane_koncowe, dtype=torch.float32)   
wynik_list = []

with torch.no_grad():
        for i, row in dane_koncowe.iterrows():  # Iteracja po wierszach DataFrame
            input_tensor = torch.tensor(row.values, dtype=torch.float32).unsqueeze(0)  # Dodajemy wymiar batcha
            output = model(input_tensor)  # Przepuszczamy przez model
            output = torch.sigmoid(output)
            predictions = (output > 0.5).float()
            print(f"🔹 Wiersz {i}: Predykcja = {predictions}")
            wynik_list.append([i+892, int(predictions)])
            print(row)    
            
wynik = pandas.DataFrame(wynik_list, columns=['PassengerId', 'Survived'])
wynik.to_csv('results.csv', index=False)

    

e:\SIUM\OneDrive - Politechnika Łódzka\[03] Studies\[98] My own\Repozytorium VSC\Kaggle\Kaggle\Titanic\scripts\preprocessing.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
e:\SIUM\OneDrive - Politechnika Łódzka\[03] Studies\[98] My own\Repozytorium VSC\Kaggle\Kaggle\Titanic\scripts\preprocessing.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change 

Test Accuracy: 0.8497
🔹 Wiersz 0: Predykcja = tensor([[1.]])
Pclass         3.0000
Sex            1.0000
Age           34.5000
SibSp          0.0000
Parch          0.0000
Fare           7.8292
Embarked_Q     1.0000
Embarked_S     0.0000
Name: 0, dtype: float64
🔹 Wiersz 1: Predykcja = tensor([[1.]])
Pclass         3.0
Sex            0.0
Age           47.0
SibSp          1.0
Parch          0.0
Fare           7.0
Embarked_Q     0.0
Embarked_S     1.0
Name: 1, dtype: float64
🔹 Wiersz 2: Predykcja = tensor([[1.]])
Pclass         2.0000
Sex            1.0000
Age           62.0000
SibSp          0.0000
Parch          0.0000
Fare           9.6875
Embarked_Q     1.0000
Embarked_S     0.0000
Name: 2, dtype: float64
🔹 Wiersz 3: Predykcja = tensor([[0.]])
Pclass         3.0000
Sex            1.0000
Age           27.0000
SibSp          0.0000
Parch          0.0000
Fare           8.6625
Embarked_Q     0.0000
Embarked_S     1.0000
Name: 3, dtype: float64
🔹 Wiersz 4: Predykcja = tensor([[1.]])
Pclass 

e:\SIUM\OneDrive - Politechnika Łódzka\[03] Studies\[98] My own\Repozytorium VSC\Kaggle\Kaggle\Titanic\scripts\preprocessing2.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
e:\SIUM\OneDrive - Politechnika Łódzka\[03] Studies\[98] My own\Repozytorium VSC\Kaggle\Kaggle\Titanic\scripts\preprocessing2.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will chang

🔹 Wiersz 242: Predykcja = tensor([[0.]])
Pclass          1.0
Sex             1.0
Age            45.0
SibSp           1.0
Parch           1.0
Fare          134.5
Embarked_Q      0.0
Embarked_S      0.0
Name: 242, dtype: float64
🔹 Wiersz 243: Predykcja = tensor([[0.]])
Pclass         3.0000
Sex            1.0000
Age           27.0000
SibSp          0.0000
Parch          0.0000
Fare           7.8875
Embarked_Q     0.0000
Embarked_S     1.0000
Name: 243, dtype: float64
🔹 Wiersz 244: Predykcja = tensor([[0.]])
Pclass         3.00
Sex            1.00
Age           27.00
SibSp          1.00
Parch          2.00
Fare          23.45
Embarked_Q     0.00
Embarked_S     1.00
Name: 244, dtype: float64
🔹 Wiersz 245: Predykcja = tensor([[0.]])
Pclass         1.0000
Sex            1.0000
Age           41.0000
SibSp          1.0000
Parch          0.0000
Fare          51.8625
Embarked_Q     0.0000
Embarked_S     1.0000
Name: 245, dtype: float64
🔹 Wiersz 246: Predykcja = tensor([[0.]])
Pclass         2.0
